# Stability Function

The Irea of this notebook is to investigate teh stability function of a Runge Kutta method with variable Weights.

The stbility function is 

$\displaystyle\sum_{i=0}^s R_i(z) b_i$ 

where $R_i(z)$ is the stability function for the Runge Kutta method with 
$ b_n = \begin{cases}
    1      & \quad \text{if } n=i\\
    0  & \quad \text{if } n \neq i
  \end{cases} $
  
 Because the stability functions are polynomials the stability functions form a $s$-dimentional vector space

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import sympy as sp


#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23')
SDIRK34 = rk.loadRKM('SDIRK34')
SDIRK54 = rk.loadRKM('SDIRK54')
TR_BDF2 = rk.loadRKM('TR-BDF2')


#Extrapolation method
ex2 = rk.extrap(2,'implicit euler')
ex3 = rk.extrap(3,'implicit euler')
ex6 = rk.extrap(6,'implicit euler')
ex8 = rk.extrap(8,'implicit euler')


In [ ]:
rkm = ex3

s = len(rkm.b)


stab_functions= ['?'] * s
b_orig = rkm.b

for i in range(s):
    b= [0]*s
    b[i] = 1
    
    rkm.b = b
    stab_functions[i] = rkm.stability_function()
    
rkm.b = b_orig



In [ ]:
def plot_amp_stab(stab_functions,n):
    p,q = stab_functions[n]

    # Convert coefficients to floats for speed
    if p.coeffs.dtype=='object':
        p = np.poly1d([float(c) for c in p.coeffs])
    if q.coeffs.dtype=='object':
        q = np.poly1d([float(c) for c in q.coeffs])
        
    u = np.linspace(-10,10,200)
    v = np.linspace(-10,10,200)
    
    U,V = np.meshgrid(u,v)
    Q = U+V*1j
    R=np.abs(p(Q)/q(Q))

    plt.pcolormesh(U, V, np.log(R),cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
    plt.colorbar()
    plt.contour(U,V,R,[0,1],colors='k',alpha=1,linewidths=3)
    plt.grid()
    
    
plot_amp_stab(stab_functions,0)

In [ ]:
plot_amp_stab(stab_functions,1)

In [ ]:
plot_amp_stab(stab_functions,2)

In [ ]:
plot_amp_stab(stab_functions,3)

In [ ]:
plot_amp_stab(stab_functions,4)

In [ ]:
plot_amp_stab(stab_functions,5)

In [ ]:
print(rkm)

For $b = (0,0,0,1,0,0)$ the stability region is a circle inside the left halfeplane. Now wo test if this is realz the case or if it is due to errors in the plotting function. 
This realy ist the case.

In [ ]:
stab_functions[3]

In [ ]:
v = np.zeros_like(b_orig)
v[3] =1

rkm.b = v
rkm.plot_stability_region()

In [ ]:
print(rkm)

The method can be writen in a more compact way. Yhis is also interesting for the case with $b = (0,1,0,0,0,0)^T$ where the stability region is the left halfeplane. Apparently for an Runge Kutta method with

\begin{array}
{c|cc}
c_1 & a\\
\hline
& 1 
\end{array}

where $c_1 = 4$ the left halve plane is in the stability region for $a \geq 1/2$ (Not 100% shure for the $=$ case)

In [ ]:
kr13 = rk.RungeKuttaMethod(np.array([[1/3]]),np.array([1]))
print(kr13)

In [ ]:
kr13.plot_stability_region()

# Testig for stability

As seen for the case $b = (0,0,0,1,0,0)$ the resulting RK-Methods are sometimes not A-stable.
Maybee it is possible to derive a condition for the b that enshures that the new method is A-Stable.

This therefor the following two conditoons have to be fullfilled: (Hairer: Solving ODE II)

$a)$ $|R(iy)|\leq1 \forall y \in R$

$b)$ $R(z)$ is analytic for $ Re(z) \leq 0$

Condition $a)$ can be tested with the E-Polynomial. To setup the E-Polynomial for an arbritrary b the sum

$$R_{sum}(iy) \sum_{i=1}^s b_i R_i(iy) $$ 
has to be writen as 
$$R_{sum}(iy) = \frac{P_{sum}(iy)}{Q_{sum}(iy)}$$

This would be easy if $Q_1 = Q_2 = \cdots = Q_{sum}$. This is true because according to the Definition 

$$R(Z) = \frac{det(I-zA+z\mathbb{1}b^T)}{det(I-zA)} $$

$Q(z)$ does only depend on $A$ and $A$ is the same for all summed Methods. The Poles can be determined by the Eigenvalues of $A$ using

$$det(I-zA) = \frac{1}{z}^s det(\frac{1}{z} I-A) = \lambda^s  det(-\lambda I+A)$$

with $$ \lambda = -\frac{1}{z}$$

Because the Matrix $A$ is triangular for Diagonally Implicit Methods the EWs are determined by the entries on the diagonal. If these are greater than zero the poles are in the right halfplane

Important: $\sum_{i=1}^s b_i = 1$ has to be fullfilled if this approch on computing the stability funtion is being used

In [ ]:
rkm = ex3

s = len(rkm.b)


stab_functions= ['?'] * s
b_orig = rkm.b

for i in range(s):
    b= [0]*s
    b[i] = 1
    
    rkm.b = b
    stab_functions[i] = rkm.stability_function()
    
rkm.b = b_orig



In [ ]:
stab_functions[0]

In [ ]:
x, y, z = sp.symbols('x y z')
expr = y*stab_functions[0][0](x)+z*stab_functions[1][0](-x)

In [ ]:
expr.simplify()

In [ ]:

#generate the b,s
variables = ''
for i in range(len(stab_functions)):
    variables = variables + 'b'+ str(i) + ' '
    
b = eval("sp.symbols('"+ variables + "')" )
    
x = sp.symbols('x')
y = sp.symbols('y',real = True)

exprp = 'b[0]*stab_functions[0][0](x)'
for i in range(1,len(stab_functions)):
    exprp = exprp+'+b['+str(i)+']*stab_functions['+str(i)+'][0](x)'

print(exprp)
P = eval(exprp)

Q = stab_functions[0][1](x)

E = Q.subs(x,sp.I*y)*Q.subs(x,-sp.I*y)-P.subs(x,sp.I*y)*P.subs(x,-sp.I*y)
E.simplify()

(b0,b1,b2,b3,b4,b5) = b

In [ ]:
sp.collect(E.expand(),y)

In [ ]:
#at y=o for arbritary b

sp.collect(E.expand(),y).subs(y,0).subs(b0,1-(b1+b2+b3+b4+b5)).simplify()

In [ ]:
# with sum(b) = 1

sp.collect(E.subs(b5,1-(b0+b1+b2+b3+b4)).simplify().expand(),y)

In [ ]:
# Case where it should be 0 everywhere

sp.collect(E.subs({b0:0,b1:1,b2:0,b3:0,b4:0,b5:0}).simplify().expand(),y)

In [ ]:
# Standard

ex3 = rk.extrap(3,'implicit euler')
print(ex3.b)

sp.collect(E.subs({b0:1/sp.S(2),b1:-2,b2:-2,b3:3/sp.S(2),b4:3/sp.S(2),b5:3/sp.S(2)}).simplify().expand(),y)

In [ ]:
ex3.stability_function()

In [ ]:
P_ref = ex3.stability_function()[0](x)

Q_ref = ex3.stability_function()[1](x)

E_ref = Q_ref.subs(x,sp.I*y)*Q_ref.subs(x,-sp.I*y)-P_ref.subs(x,sp.I*y)*P_ref.subs(x,-sp.I*y)
sp.collect(E_ref.simplify().expand(),y)